In [1]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
path = str(Path.cwd().parent)
print(path)
sys.path.insert(1, path)

import numpy as np
import pandas as pd

c:\Users\jaesc2\GitHub\skforecast


In [13]:
import keras
import numpy as np
import pandas as pd

from skforecast.deep_learning import ForecasterRnn
from skforecast.deep_learning.utils import create_and_compile_model, create_and_compile_model_exog

# set backend torch
import os
os.environ["KERAS_BACKEND"] = "torch"

series = pd.DataFrame(
    {
        "1": pd.Series(np.arange(50.)),
        "2": pd.Series(np.arange(50., 100.)),
        "3": pd.Series(np.arange(100., 150.)),
    }
)

exog = pd.DataFrame(
    {
        "exog_1": pd.Series(np.arange(50.)),
        "exog_2": pd.Series(np.arange(50., 100.)),
        "exog_3": pd.Series(np.arange(100., 150.)),
    }
)

exog_predict = pd.DataFrame(
    {
        "exog_1": pd.Series(np.arange(50., 60)),
        "exog_2": pd.Series(np.arange(100., 110.)),
        "exog_3": pd.Series(np.arange(150., 160.)),
    },
    index=pd.RangeIndex(start=50, stop=60, step=1)
)


lags = 4
steps = 5
levels = ["1", "2"]
activation = "relu"
optimizer = keras.optimizers.Adam(learning_rate=0.01)
loss = keras.losses.MeanSquaredError()
recurrent_units = 100
dense_units = [128, 64]


# model = create_and_compile_model(
#     series=series,
#     lags=lags,
#     steps=steps,
#     levels=levels,
#     recurrent_units=recurrent_units,
#     dense_units=dense_units,
#     activation=activation,
#     optimizer=optimizer,
#     loss=loss,
# )

model = create_and_compile_model_exog(
    series=series,
    lags=lags,
    steps=steps,
    levels=levels,
    exog=exog,
    recurrent_units=recurrent_units,
    dense_units=dense_units,
    activation=activation,
    optimizer=optimizer,
    loss=loss,
)
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, 4, 3)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_1 (LSTM)       │ (None, 100)       │     41,600 │ input_layer_2[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_1 (Flatten) │ (None, 100)       │          0 │ lstm_1[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_3       │ (None, 3)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_1       │ (None, 103)       │          0 │ flatten_1[0][0],  │
│ (Concatenate)       │                   │            │ input_layer_3[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 128)       │     13,312 │ concatenate_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 64)        │      8,256 │ dense_3[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_5 (Dense)     │ (None, 10)        │        650 │ dense_4[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape_1 (Reshape) │ (None, 5, 2)      │          0 │ dense_5[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 63,818 (249.29 KB)

 Trainable params: 63,818 (249.29 KB)

 Non-trainable params: 0 (0.00 B)

In [14]:
forecaster = ForecasterRnn(model, levels, lags=lags)

c:\Users\jaesc2\Miniconda3\envs\skforecast_py11_2\Lib\site-packages\keras\src\saving\saving_lib.py:719: UserWarning: Skipping variable loading for optimizer 'adam', because it has 20 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [15]:
forecaster.fit(
    series=series, exog=exog, store_in_sample_residuals=True
)

c:\Users\jaesc2\Miniconda3\envs\skforecast_py11_2\Lib\site-packages\keras\src\models\functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['input_layer_2', 'input_layer_3']. Received: the structure of inputs=('*', '*')
  warnings.warn(


2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - loss: 0.3123


c:\Users\jaesc2\Miniconda3\envs\skforecast_py11_2\Lib\site-packages\keras\src\models\functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['input_layer_2', 'input_layer_3']. Received: the structure of inputs=('*', '*')
  warnings.warn(


In [16]:
import keras

print(keras.__version__)
print(keras.backend.backend())

3.6.0
tensorflow


In [17]:
_ = forecaster.create_train_X_y(series)
_[0].shape

(42, 4, 3)

In [18]:
forecaster.in_sample_residuals_[1]

array([[-2.22314281e-02, -5.04292456e-02],
       [-1.28548702e-02, -4.66091444e-02],
       [-3.63974395e-03, -4.22462982e-02],
       [ 5.81763046e-03, -3.90866016e-02],
       [ 1.48060206e-02, -3.57131776e-02],
       [ 2.25592535e-02, -3.32082677e-02],
       [ 2.97136088e-02, -3.05801402e-02],
       [ 3.68845341e-02, -2.80100825e-02],
       [ 4.38576614e-02, -2.54078532e-02],
       [ 5.06460740e-02, -2.27991866e-02],
       [ 5.74173800e-02, -2.02286669e-02],
       [ 6.41956001e-02, -1.76781148e-02],
       [ 7.11251862e-02, -1.52867668e-02],
       [ 7.83910021e-02, -1.30097404e-02],
       [ 8.56444501e-02, -1.07622779e-02],
       [ 9.28966166e-02, -8.56226074e-03],
       [ 1.00123093e-01, -6.38703911e-03],
       [ 1.07333000e-01, -4.23601695e-03],
       [ 1.14492750e-01, -2.13196326e-03],
       [ 1.21629730e-01, -6.80831014e-05],
       [ 1.28744061e-01,  1.97284927e-03],
       [ 1.35840927e-01,  3.98958216e-03],
       [ 1.42717316e-01,  5.94069033e-03],
       [ 1.

In [19]:
forecaster.in_sample_residuals_[2][:5, :]

array([[-0.13833536,  0.0371483 ],
       [-0.14441836,  0.05066295],
       [-0.15144756,  0.06424513],
       [-0.15945001,  0.07783952],
       [-0.16647619,  0.09219622]])

In [20]:
forecaster.series_names_in_

['1', '2', '3']

In [21]:
forecaster.levels

['1', '2']

In [22]:
forecaster.last_window_

,1,2,3
46,46.0,96.0,146.0
47,47.0,97.0,147.0
48,48.0,98.0,148.0
49,49.0,99.0,149.0


In [23]:
inputs = forecaster._create_predict_inputs(
    steps=3, levels=['1', '2'], exog=exog_predict
)
inputs

╭──────────────────────────────── MissingValuesWarning ────────────────────────────────╮
│ `exog` has missing values. Most of machine learning models do not allow missing      │
│ values. Prediction method may fail.                                                  │
│                                                                                      │
│ Category : MissingValuesWarning                                                      │
│ Location : c:\Users\jaesc2\GitHub\skforecast\skforecast\utils\utils.py:1046          │
│ Suppress : warnings.simplefilter('ignore', category=MissingValuesWarning)            │
╰──────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────── MissingValuesWarning ────────────────────────────────╮
│ `exog` has missing values. Most machine learning models do not allow missing values. │
│ Fitting the forecaster may fail.                                                     │
│                                                                                      │
│ Category : MissingValuesWarning                                                      │
│ Location : c:\Users\jaesc2\GitHub\skforecast\skforecast\utils\utils.py:568           │
│ Suppress : warnings.simplefilter('ignore', category=MissingValuesWarning)            │
╰──────────────────────────────────────────────────────────────────────────────────────╯

TypeError: slice indices must be integers or None or have an __index__ method

In [36]:
predictions = forecaster.predict(steps=2, levels=['1', "2"])
predictions

,level,pred
10,1,0.512350
10,2,50.650616
11,1,0.694859
11,2,50.684875


In [34]:
_[0]

array([[[0.        , 0.        , 0.        ],
        [0.11111111, 0.11111111, 0.11111111],
        [0.22222222, 0.22222222, 0.22222222]],

       [[0.11111111, 0.11111111, 0.11111111],
        [0.22222222, 0.22222222, 0.22222222],
        [0.33333333, 0.33333333, 0.33333333]],

       [[0.22222222, 0.22222222, 0.22222222],
        [0.33333333, 0.33333333, 0.33333333],
        [0.44444444, 0.44444444, 0.44444444]],

       [[0.33333333, 0.33333333, 0.33333333],
        [0.44444444, 0.44444444, 0.44444444],
        [0.55555556, 0.55555556, 0.55555556]],

       [[0.44444444, 0.44444444, 0.44444444],
        [0.55555556, 0.55555556, 0.55555556],
        [0.66666667, 0.66666667, 0.66666667]],

       [[0.55555556, 0.55555556, 0.55555556],
        [0.66666667, 0.66666667, 0.66666667],
        [0.77777778, 0.77777778, 0.77777778]]])

In [ ]:
steps = 2
levels = 2

_ = forecaster.create_train_X_y(series)
forecaster.regressor.predict(x=_[0], verbose=0)

array([[[0.02714103, 0.03129969],
        [0.03067968, 0.02870654]],

       [[0.03108143, 0.03507841],
        [0.03619017, 0.03491943]],

       [[0.03488293, 0.03987598],
        [0.04253578, 0.04175308]],

       [[0.03886062, 0.04558587],
        [0.04964645, 0.04881068]],

       [[0.04309836, 0.05186733],
        [0.056449  , 0.05565476]],

       [[0.04744031, 0.05885262],
        [0.06324376, 0.06249271]]], dtype=float32)

In [ ]:
_ = forecaster.create_train_X_y(series)
forecaster.regressor.predict(x=_[0], verbose=0)

In [125]:
predictions = forecaster.predict(steps=3, levels=None)
predictions

,level,pred
50,1,7.713717
50,2,61.483234
51,1,25.941444
51,2,55.504932
52,1,31.411823
52,2,82.724594


In [9]:
predictions = forecaster._predict_interval_conformal(steps=3, levels=None)
predictions

[[0.55023268 0.45024333]
 [0.58160202 0.35509963]
 [0.41151223 0.49386649]]


,level,pred,lower_bound,upper_bound
50,1,18.687877,-8.273525,45.649278
50,2,74.351726,52.289802,96.413649
51,1,18.113966,-10.384533,46.612465
51,2,80.556508,63.156627,97.956390
52,1,28.626856,8.462757,48.790955
52,2,74.064660,49.865202,98.264118


In [140]:
predictions = forecaster._predict_interval_conformal(steps=3, levels=['1'])
predictions

,level,pred,lower_bound,upper_bound
50,1,7.713717,-29.086494,44.513928
51,1,25.941444,4.372634,47.510253
52,1,31.411823,13.783603,49.040043


In [144]:
np.quantile(
    np.abs(forecaster.in_sample_residuals_[2][:, 1]), 0.95
)

np.float64(0.8132576974450934)

In [ ]:
forecaster.in_sample_residuals_[1]

{1: array([[ 0.00300063, -0.02063499],
        [ 0.02164905, -0.00324193],
        [ 0.04019636,  0.01412207],
        [ 0.05871693,  0.03156052],
        [ 0.07703444,  0.04902489],
        [ 0.09533564,  0.06641259],
        [ 0.11370614,  0.0837122 ],
        [ 0.13206353,  0.10091294],
        [ 0.15043704,  0.11811714],
        [ 0.1688066 ,  0.1353427 ],
        [ 0.1871703 ,  0.15255236],
        [ 0.20552565,  0.16975107],
        [ 0.2238739 ,  0.18696467],
        [ 0.2422113 ,  0.20416079],
        [ 0.26053225,  0.22133179],
        [ 0.2789068 ,  0.23851671],
        [ 0.29727427,  0.2556899 ],
        [ 0.31563729,  0.27285653],
        [ 0.33399589,  0.29001659],
        [ 0.35235008,  0.30717007],
        [ 0.37069992,  0.32431521],
        [ 0.38904568,  0.34144994],
        [ 0.40738741,  0.3585762 ],
        [ 0.42566772,  0.37572525],
        [ 0.44391787,  0.39288245],
        [ 0.46216279,  0.41003336],
        [ 0.48040247,  0.42717802],
        [ 0.49863687,  0.